In [1631]:
import pandas as pd
import numpy as np
import holidays
import math

In [1632]:
"""
url = "https://data.cityofnewyork.us/resource/i4gi-tjb9.csv"

df = pd.read_csv(url)

df.head()
"""

'\nurl = "https://data.cityofnewyork.us/resource/i4gi-tjb9.csv"\n\ndf = pd.read_csv(url)\n\ndf.head()\n'

In [1633]:
df.columns

Index(['id', 'speed', 'travel_time', 'status', 'data_as_of', 'link_id',
       'link_points', 'encoded_poly_line', 'encoded_poly_line_lvls', 'owner',
       'transcom_id', 'borough', 'link_name'],
      dtype='object')

**Investigating The Status Column**

In [1634]:
df['status'].unique()

array([   0, -101])

In [1635]:
df[df['status'] == -101]['speed'].value_counts()

,count
speed,
0.00,181
35.41,3
4.34,2
9.32,2
13.04,1
28.58,1
3.72,1
27.96,1
37.90,1


In [1636]:
df[df['status'] == -101]['travel_time'].value_counts()

,count
travel_time,
0,181
594,2
238,1
1865,1
1348,1
768,1
242,1
192,1
286,1


**Removing Unnecessary Columns**

In [1637]:
data = df.drop(['id', 'encoded_poly_line', 'encoded_poly_line_lvls', 'owner', 'link_id', 'transcom_id'], axis = 1)
data.head()

,speed,travel_time,status,data_as_of,link_points,borough,link_name
0,18.01,855,0,2025-03-19T08:43:10.000,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE
1,15.53,688,0,2025-03-19T08:43:10.000,"40.67445,-73.80181 40.67374,-73.80154 40.67261...",Queens,VWE S MP2.66 (Exit 2 Rockaway Blvd) - Belt Pkw...
2,0.00,0,-101,2025-03-19T08:43:10.000,"40.6167105,-74.15242 40.61572,-74.15276 40.614...",Staten Island,SIE W - MLK N WOOLEY AVENUE - WLAKER STREET
3,8.69,1358,0,2025-03-19T08:43:10.000,"40.66673,-73.78649 40.66642,-73.78958 40.66642...",Queens,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave
4,19.88,448,0,2025-03-19T08:43:10.000,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...


In [1638]:
new_data = data[data['status'] == -101]
new_data

,speed,travel_time,status,data_as_of,link_points,borough,link_name
2,0.0,0,-101,2025-03-19T08:43:10.000,"40.6167105,-74.15242 40.61572,-74.15276 40.614...",Staten Island,SIE W - MLK N WOOLEY AVENUE - WLAKER STREET
16,0.0,0,-101,2025-03-19T08:43:10.000,"40.63092,-74.14592 40.62975,-74.14593 40.62877...",Staten Island,MLK S - SIE E WALKER STREET - WOOLEY AVENUE
21,0.0,0,-101,2025-03-19T08:43:10.000,"40.63089,-74.14569 40.6298,-74.14569 40.62883,...",Staten Island,SIE E-MLK N RICHMOND AVENUE - WALKER STREET
22,0.0,0,-101,2025-03-19T08:43:10.000,"40.77158,-73.994441 40.7713004,-73.99455 40.77...",Manhattan,12th Ave S 57th St - 45th St
23,0.0,0,-101,2025-03-19T08:43:10.000,"40.74047,-74.009251 40.74137,-74.00893 40.7431...",Manhattan,11th ave n ganservoort - 12th ave @ 40th st
...,...,...,...,...,...,...,...
977,0.0,0,-101,2025-03-19T08:03:11.000,"40.7607,-74.002141 40.76212,-74.91 40.76335,-7...",Manhattan,12th Ave N 40th - 57th St
984,0.0,0,-101,2025-03-19T08:03:11.000,"40.76375,-73.999191 40.763521,-73.99935 40.762...",Manhattan,12th ave @ 45th - 11 ave ganservoort st
988,0.0,0,-101,2025-03-19T08:03:11.000,"40.65632,-74.11972 40.655041,-74.12241 40.6544...",Staten Island,ROUTE 169 S - MLK WALKER STREET
989,0.0,0,-101,2025-03-19T08:03:11.000,"40.77158,-73.994441 40.7713004,-73.99455 40.77...",Manhattan,12th Ave S 57th St - 45th St


In [1639]:
data.isna().sum()

,0
speed,0
travel_time,0
status,0
data_as_of,0
link_points,0
borough,0
link_name,0


In [1640]:
print(data['status'].value_counts())
print(len(data[~((data['speed'] == 0) & (data['travel_time'] == 0)) & (data['status'] == -101)]))

status
 0      805
-101    195
Name: count, dtype: int64
14


In [1641]:
data.loc[(data['speed'] == 0) & (data['travel_time'] == 0), 'speed'] = np.nan
data.loc[(data['speed'].isna()) & (data['travel_time'] == 0), 'travel_time'] = np.nan

In [1642]:
data.isna().sum()

,0
speed,181
travel_time,181
status,0
data_as_of,0
link_points,0
borough,0
link_name,0


**Filling Invalid Rows with Forward & Backward Filling**

In [1643]:
data['speed_filled'] = data['speed'].fillna(method='ffill')
data['travel_time_filled'] = data['travel_time'].fillna(method='ffill')

data['speed'] = data['speed_filled'].fillna(method='bfill')
data['travel_time'] = data['travel_time_filled'].fillna(method='bfill')

<ipython-input-1643-9c1995c98dd1>:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['speed_filled'] = data['speed'].fillna(method='ffill')
<ipython-input-1643-9c1995c98dd1>:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['travel_time_filled'] = data['travel_time'].fillna(method='ffill')
<ipython-input-1643-9c1995c98dd1>:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['speed'] = data['speed_filled'].fillna(method='bfill')
<ipython-input-1643-9c1995c98dd1>:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['travel_time'] = data['travel_time_filled'].fillna(method='bfill')


In [1644]:
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled
0,18.01,855.0,0,2025-03-19T08:43:10.000,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,18.01,855.0
1,15.53,688.0,0,2025-03-19T08:43:10.000,"40.67445,-73.80181 40.67374,-73.80154 40.67261...",Queens,VWE S MP2.66 (Exit 2 Rockaway Blvd) - Belt Pkw...,15.53,688.0
2,15.53,688.0,-101,2025-03-19T08:43:10.000,"40.6167105,-74.15242 40.61572,-74.15276 40.614...",Staten Island,SIE W - MLK N WOOLEY AVENUE - WLAKER STREET,15.53,688.0
3,8.69,1358.0,0,2025-03-19T08:43:10.000,"40.66673,-73.78649 40.66642,-73.78958 40.66642...",Queens,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave,8.69,1358.0
4,19.88,448.0,0,2025-03-19T08:43:10.000,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,19.88,448.0
...,...,...,...,...,...,...,...,...,...
995,18.01,490.0,0,2025-03-19T08:03:11.000,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,18.01,490.0
996,50.95,28.0,0,2025-03-19T08:03:11.000,"40.6077805,-74.14091 40.60826,-74.132101",Staten Island,SIE W BRADLEY AVENUE - WOOLEY AVENUE,50.95,28.0
997,14.91,1023.0,0,2025-03-19T08:03:11.000,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,14.91,1023.0
998,14.91,1023.0,-101,2025-03-19T08:03:11.000,"40.81376,-73.93128 40.8127206,-73.9316 40.8121...",Bronx,MDE N RFK Bridge - 142nd St,14.91,1023.0


In [1645]:
print(data[data['status'] == -101][['speed', 'travel_time']])

     speed  travel_time
2    15.53        688.0
16   32.31         49.0
21   43.49        106.0
22   43.49        106.0
23   43.49        106.0
..     ...          ...
977   8.69       1072.0
984  19.26        125.0
988  49.08         57.0
989  49.08         57.0
998  14.91       1023.0

[195 rows x 2 columns]


In [1646]:
print(len(data[(data['speed'] == 0) & (data['travel_time'] == 0)]))

0


**Creating Hour and Minute Columns**

In [1647]:
data['data_as_of'] = pd.to_datetime(data['data_as_of'])

data['hour'] = data['data_as_of'].dt.hour
data['minute'] = data['data_as_of'].dt.minute

**Combining The Hour and Minute Column and Encoding It**

In [1648]:
data['minute_number'] = data['hour'] * 60 + data['minute']
data['minute_sin'] = np.sin(2 * np.pi * data['minute_number'] / (60*24))
data['minute_cos'] = np.cos(2 * np.pi * data['minute_number'] / (60*24))

In [1649]:
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,minute,minute_number,minute_sin,minute_cos
0,18.01,855.0,0,2025-03-19 08:43:10,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,18.01,855.0,8,43,523,0.757565,-0.652760
1,15.53,688.0,0,2025-03-19 08:43:10,"40.67445,-73.80181 40.67374,-73.80154 40.67261...",Queens,VWE S MP2.66 (Exit 2 Rockaway Blvd) - Belt Pkw...,15.53,688.0,8,43,523,0.757565,-0.652760
2,15.53,688.0,-101,2025-03-19 08:43:10,"40.6167105,-74.15242 40.61572,-74.15276 40.614...",Staten Island,SIE W - MLK N WOOLEY AVENUE - WLAKER STREET,15.53,688.0,8,43,523,0.757565,-0.652760
3,8.69,1358.0,0,2025-03-19 08:43:10,"40.66673,-73.78649 40.66642,-73.78958 40.66642...",Queens,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave,8.69,1358.0,8,43,523,0.757565,-0.652760
4,19.88,448.0,0,2025-03-19 08:43:10,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,19.88,448.0,8,43,523,0.757565,-0.652760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,18.01,490.0,0,2025-03-19 08:03:11,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,18.01,490.0,8,3,483,0.859406,-0.511293
996,50.95,28.0,0,2025-03-19 08:03:11,"40.6077805,-74.14091 40.60826,-74.132101",Staten Island,SIE W BRADLEY AVENUE - WOOLEY AVENUE,50.95,28.0,8,3,483,0.859406,-0.511293
997,14.91,1023.0,0,2025-03-19 08:03:11,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,14.91,1023.0,8,3,483,0.859406,-0.511293
998,14.91,1023.0,-101,2025-03-19 08:03:11,"40.81376,-73.93128 40.8127206,-73.9316 40.8121...",Bronx,MDE N RFK Bridge - 142nd St,14.91,1023.0,8,3,483,0.859406,-0.511293


**Creating Day Name Column**

In [1650]:
data['day_of_week'] = data['data_as_of'].dt.day_name()
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,minute,minute_number,minute_sin,minute_cos,day_of_week
0,18.01,855.0,0,2025-03-19 08:43:10,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,18.01,855.0,8,43,523,0.757565,-0.652760,Wednesday
1,15.53,688.0,0,2025-03-19 08:43:10,"40.67445,-73.80181 40.67374,-73.80154 40.67261...",Queens,VWE S MP2.66 (Exit 2 Rockaway Blvd) - Belt Pkw...,15.53,688.0,8,43,523,0.757565,-0.652760,Wednesday
2,15.53,688.0,-101,2025-03-19 08:43:10,"40.6167105,-74.15242 40.61572,-74.15276 40.614...",Staten Island,SIE W - MLK N WOOLEY AVENUE - WLAKER STREET,15.53,688.0,8,43,523,0.757565,-0.652760,Wednesday
3,8.69,1358.0,0,2025-03-19 08:43:10,"40.66673,-73.78649 40.66642,-73.78958 40.66642...",Queens,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave,8.69,1358.0,8,43,523,0.757565,-0.652760,Wednesday
4,19.88,448.0,0,2025-03-19 08:43:10,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,19.88,448.0,8,43,523,0.757565,-0.652760,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,18.01,490.0,0,2025-03-19 08:03:11,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,18.01,490.0,8,3,483,0.859406,-0.511293,Wednesday
996,50.95,28.0,0,2025-03-19 08:03:11,"40.6077805,-74.14091 40.60826,-74.132101",Staten Island,SIE W BRADLEY AVENUE - WOOLEY AVENUE,50.95,28.0,8,3,483,0.859406,-0.511293,Wednesday
997,14.91,1023.0,0,2025-03-19 08:03:11,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,14.91,1023.0,8,3,483,0.859406,-0.511293,Wednesday
998,14.91,1023.0,-101,2025-03-19 08:03:11,"40.81376,-73.93128 40.8127206,-73.9316 40.8121...",Bronx,MDE N RFK Bridge - 142nd St,14.91,1023.0,8,3,483,0.859406,-0.511293,Wednesday


**Encoding The Days Column**

In [1651]:
days = {"Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3, "Friday": 4, "Saturday": 5, "Sunday": 6}
data['day_of_week_num'] = data['day_of_week'].map(days)
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,minute,minute_number,minute_sin,minute_cos,day_of_week,day_of_week_num
0,18.01,855.0,0,2025-03-19 08:43:10,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,18.01,855.0,8,43,523,0.757565,-0.652760,Wednesday,2
1,15.53,688.0,0,2025-03-19 08:43:10,"40.67445,-73.80181 40.67374,-73.80154 40.67261...",Queens,VWE S MP2.66 (Exit 2 Rockaway Blvd) - Belt Pkw...,15.53,688.0,8,43,523,0.757565,-0.652760,Wednesday,2
2,15.53,688.0,-101,2025-03-19 08:43:10,"40.6167105,-74.15242 40.61572,-74.15276 40.614...",Staten Island,SIE W - MLK N WOOLEY AVENUE - WLAKER STREET,15.53,688.0,8,43,523,0.757565,-0.652760,Wednesday,2
3,8.69,1358.0,0,2025-03-19 08:43:10,"40.66673,-73.78649 40.66642,-73.78958 40.66642...",Queens,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave,8.69,1358.0,8,43,523,0.757565,-0.652760,Wednesday,2
4,19.88,448.0,0,2025-03-19 08:43:10,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,19.88,448.0,8,43,523,0.757565,-0.652760,Wednesday,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,18.01,490.0,0,2025-03-19 08:03:11,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,18.01,490.0,8,3,483,0.859406,-0.511293,Wednesday,2
996,50.95,28.0,0,2025-03-19 08:03:11,"40.6077805,-74.14091 40.60826,-74.132101",Staten Island,SIE W BRADLEY AVENUE - WOOLEY AVENUE,50.95,28.0,8,3,483,0.859406,-0.511293,Wednesday,2
997,14.91,1023.0,0,2025-03-19 08:03:11,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,14.91,1023.0,8,3,483,0.859406,-0.511293,Wednesday,2
998,14.91,1023.0,-101,2025-03-19 08:03:11,"40.81376,-73.93128 40.8127206,-73.9316 40.8121...",Bronx,MDE N RFK Bridge - 142nd St,14.91,1023.0,8,3,483,0.859406,-0.511293,Wednesday,2


In [1652]:
data['day_sin'] = np.sin(2 * np.pi * data['day_of_week_num'] / 7)
data['day_cos'] = np.cos(2 * np.pi * data['day_of_week_num'] / 7)

In [1653]:
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,minute,minute_number,minute_sin,minute_cos,day_of_week,day_of_week_num,day_sin,day_cos
0,18.01,855.0,0,2025-03-19 08:43:10,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,18.01,855.0,8,43,523,0.757565,-0.652760,Wednesday,2,0.974928,-0.222521
1,15.53,688.0,0,2025-03-19 08:43:10,"40.67445,-73.80181 40.67374,-73.80154 40.67261...",Queens,VWE S MP2.66 (Exit 2 Rockaway Blvd) - Belt Pkw...,15.53,688.0,8,43,523,0.757565,-0.652760,Wednesday,2,0.974928,-0.222521
2,15.53,688.0,-101,2025-03-19 08:43:10,"40.6167105,-74.15242 40.61572,-74.15276 40.614...",Staten Island,SIE W - MLK N WOOLEY AVENUE - WLAKER STREET,15.53,688.0,8,43,523,0.757565,-0.652760,Wednesday,2,0.974928,-0.222521
3,8.69,1358.0,0,2025-03-19 08:43:10,"40.66673,-73.78649 40.66642,-73.78958 40.66642...",Queens,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave,8.69,1358.0,8,43,523,0.757565,-0.652760,Wednesday,2,0.974928,-0.222521
4,19.88,448.0,0,2025-03-19 08:43:10,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,19.88,448.0,8,43,523,0.757565,-0.652760,Wednesday,2,0.974928,-0.222521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,18.01,490.0,0,2025-03-19 08:03:11,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,18.01,490.0,8,3,483,0.859406,-0.511293,Wednesday,2,0.974928,-0.222521
996,50.95,28.0,0,2025-03-19 08:03:11,"40.6077805,-74.14091 40.60826,-74.132101",Staten Island,SIE W BRADLEY AVENUE - WOOLEY AVENUE,50.95,28.0,8,3,483,0.859406,-0.511293,Wednesday,2,0.974928,-0.222521
997,14.91,1023.0,0,2025-03-19 08:03:11,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,14.91,1023.0,8,3,483,0.859406,-0.511293,Wednesday,2,0.974928,-0.222521
998,14.91,1023.0,-101,2025-03-19 08:03:11,"40.81376,-73.93128 40.8127206,-73.9316 40.8121...",Bronx,MDE N RFK Bridge - 142nd St,14.91,1023.0,8,3,483,0.859406,-0.511293,Wednesday,2,0.974928,-0.222521


**Creating and Encoding The Week Column**

In [1654]:
data['week_of_year'] = data['data_as_of'].dt.isocalendar().week
data['week_num_sin'] = np.sin(2 * np.pi * data['week_of_year'] / 53)
data['week_num_cos'] = np.cos(2 * np.pi * data['week_of_year'] / 53)

In [1655]:
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,...,minute_number,minute_sin,minute_cos,day_of_week,day_of_week_num,day_sin,day_cos,week_of_year,week_num_sin,week_num_cos
0,18.01,855.0,0,2025-03-19 08:43:10,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,18.01,855.0,8,...,523,0.757565,-0.652760,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647
1,15.53,688.0,0,2025-03-19 08:43:10,"40.67445,-73.80181 40.67374,-73.80154 40.67261...",Queens,VWE S MP2.66 (Exit 2 Rockaway Blvd) - Belt Pkw...,15.53,688.0,8,...,523,0.757565,-0.652760,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647
2,15.53,688.0,-101,2025-03-19 08:43:10,"40.6167105,-74.15242 40.61572,-74.15276 40.614...",Staten Island,SIE W - MLK N WOOLEY AVENUE - WLAKER STREET,15.53,688.0,8,...,523,0.757565,-0.652760,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647
3,8.69,1358.0,0,2025-03-19 08:43:10,"40.66673,-73.78649 40.66642,-73.78958 40.66642...",Queens,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave,8.69,1358.0,8,...,523,0.757565,-0.652760,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647
4,19.88,448.0,0,2025-03-19 08:43:10,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,19.88,448.0,8,...,523,0.757565,-0.652760,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,18.01,490.0,0,2025-03-19 08:03:11,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,18.01,490.0,8,...,483,0.859406,-0.511293,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647
996,50.95,28.0,0,2025-03-19 08:03:11,"40.6077805,-74.14091 40.60826,-74.132101",Staten Island,SIE W BRADLEY AVENUE - WOOLEY AVENUE,50.95,28.0,8,...,483,0.859406,-0.511293,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647
997,14.91,1023.0,0,2025-03-19 08:03:11,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,14.91,1023.0,8,...,483,0.859406,-0.511293,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647
998,14.91,1023.0,-101,2025-03-19 08:03:11,"40.81376,-73.93128 40.8127206,-73.9316 40.8121...",Bronx,MDE N RFK Bridge - 142nd St,14.91,1023.0,8,...,483,0.859406,-0.511293,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647


**Creating and Encoding The Month Column**

In [1656]:
data['month'] = data['data_as_of'].dt.month_name()
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,...,minute_sin,minute_cos,day_of_week,day_of_week_num,day_sin,day_cos,week_of_year,week_num_sin,week_num_cos,month
0,18.01,855.0,0,2025-03-19 08:43:10,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,18.01,855.0,8,...,0.757565,-0.652760,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March
1,15.53,688.0,0,2025-03-19 08:43:10,"40.67445,-73.80181 40.67374,-73.80154 40.67261...",Queens,VWE S MP2.66 (Exit 2 Rockaway Blvd) - Belt Pkw...,15.53,688.0,8,...,0.757565,-0.652760,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March
2,15.53,688.0,-101,2025-03-19 08:43:10,"40.6167105,-74.15242 40.61572,-74.15276 40.614...",Staten Island,SIE W - MLK N WOOLEY AVENUE - WLAKER STREET,15.53,688.0,8,...,0.757565,-0.652760,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March
3,8.69,1358.0,0,2025-03-19 08:43:10,"40.66673,-73.78649 40.66642,-73.78958 40.66642...",Queens,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave,8.69,1358.0,8,...,0.757565,-0.652760,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March
4,19.88,448.0,0,2025-03-19 08:43:10,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,19.88,448.0,8,...,0.757565,-0.652760,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,18.01,490.0,0,2025-03-19 08:03:11,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,18.01,490.0,8,...,0.859406,-0.511293,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March
996,50.95,28.0,0,2025-03-19 08:03:11,"40.6077805,-74.14091 40.60826,-74.132101",Staten Island,SIE W BRADLEY AVENUE - WOOLEY AVENUE,50.95,28.0,8,...,0.859406,-0.511293,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March
997,14.91,1023.0,0,2025-03-19 08:03:11,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,14.91,1023.0,8,...,0.859406,-0.511293,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March
998,14.91,1023.0,-101,2025-03-19 08:03:11,"40.81376,-73.93128 40.8127206,-73.9316 40.8121...",Bronx,MDE N RFK Bridge - 142nd St,14.91,1023.0,8,...,0.859406,-0.511293,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March


In [1657]:
months = {"January": 1, "February": 2, "March": 3,
            "April": 4, "May": 5, "June": 6,
            "July": 7, "August": 8, "September": 9,
            "October": 10, "November": 11, "December": 12}

data['month_num'] = data['month'].map(months)

In [1658]:
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,...,minute_cos,day_of_week,day_of_week_num,day_sin,day_cos,week_of_year,week_num_sin,week_num_cos,month,month_num
0,18.01,855.0,0,2025-03-19 08:43:10,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,18.01,855.0,8,...,-0.652760,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March,3
1,15.53,688.0,0,2025-03-19 08:43:10,"40.67445,-73.80181 40.67374,-73.80154 40.67261...",Queens,VWE S MP2.66 (Exit 2 Rockaway Blvd) - Belt Pkw...,15.53,688.0,8,...,-0.652760,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March,3
2,15.53,688.0,-101,2025-03-19 08:43:10,"40.6167105,-74.15242 40.61572,-74.15276 40.614...",Staten Island,SIE W - MLK N WOOLEY AVENUE - WLAKER STREET,15.53,688.0,8,...,-0.652760,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March,3
3,8.69,1358.0,0,2025-03-19 08:43:10,"40.66673,-73.78649 40.66642,-73.78958 40.66642...",Queens,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave,8.69,1358.0,8,...,-0.652760,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March,3
4,19.88,448.0,0,2025-03-19 08:43:10,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,19.88,448.0,8,...,-0.652760,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,18.01,490.0,0,2025-03-19 08:03:11,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,18.01,490.0,8,...,-0.511293,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March,3
996,50.95,28.0,0,2025-03-19 08:03:11,"40.6077805,-74.14091 40.60826,-74.132101",Staten Island,SIE W BRADLEY AVENUE - WOOLEY AVENUE,50.95,28.0,8,...,-0.511293,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March,3
997,14.91,1023.0,0,2025-03-19 08:03:11,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,14.91,1023.0,8,...,-0.511293,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March,3
998,14.91,1023.0,-101,2025-03-19 08:03:11,"40.81376,-73.93128 40.8127206,-73.9316 40.8121...",Bronx,MDE N RFK Bridge - 142nd St,14.91,1023.0,8,...,-0.511293,Wednesday,2,0.974928,-0.222521,12,0.98904,0.147647,March,3


In [1659]:
data['month_sin'] = np.sin(2 * np.pi * data['month_num'] / 12)
data['month_cos'] = np.cos(2 * np.pi * data['month_num'] / 12)

In [1660]:
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,...,day_of_week_num,day_sin,day_cos,week_of_year,week_num_sin,week_num_cos,month,month_num,month_sin,month_cos
0,18.01,855.0,0,2025-03-19 08:43:10,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,18.01,855.0,8,...,2,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17
1,15.53,688.0,0,2025-03-19 08:43:10,"40.67445,-73.80181 40.67374,-73.80154 40.67261...",Queens,VWE S MP2.66 (Exit 2 Rockaway Blvd) - Belt Pkw...,15.53,688.0,8,...,2,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17
2,15.53,688.0,-101,2025-03-19 08:43:10,"40.6167105,-74.15242 40.61572,-74.15276 40.614...",Staten Island,SIE W - MLK N WOOLEY AVENUE - WLAKER STREET,15.53,688.0,8,...,2,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17
3,8.69,1358.0,0,2025-03-19 08:43:10,"40.66673,-73.78649 40.66642,-73.78958 40.66642...",Queens,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave,8.69,1358.0,8,...,2,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17
4,19.88,448.0,0,2025-03-19 08:43:10,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,19.88,448.0,8,...,2,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,18.01,490.0,0,2025-03-19 08:03:11,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,18.01,490.0,8,...,2,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17
996,50.95,28.0,0,2025-03-19 08:03:11,"40.6077805,-74.14091 40.60826,-74.132101",Staten Island,SIE W BRADLEY AVENUE - WOOLEY AVENUE,50.95,28.0,8,...,2,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17
997,14.91,1023.0,0,2025-03-19 08:03:11,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,14.91,1023.0,8,...,2,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17
998,14.91,1023.0,-101,2025-03-19 08:03:11,"40.81376,-73.93128 40.8127206,-73.9316 40.8121...",Bronx,MDE N RFK Bridge - 142nd St,14.91,1023.0,8,...,2,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17


**Creating a Holidays Column (Binary)**

In [1661]:
us_holidays = holidays.US(years=data['data_as_of'].dt.year.unique())
us_holidays

{datetime.date(2025, 1, 1): "New Year's Day", datetime.date(2025, 5, 26): 'Memorial Day', datetime.date(2025, 6, 19): 'Juneteenth National Independence Day', datetime.date(2025, 7, 4): 'Independence Day', datetime.date(2025, 9, 1): 'Labor Day', datetime.date(2025, 11, 11): 'Veterans Day', datetime.date(2025, 11, 27): 'Thanksgiving', datetime.date(2025, 12, 25): 'Christmas Day', datetime.date(2025, 1, 20): 'Martin Luther King Jr. Day', datetime.date(2025, 2, 17): "Washington's Birthday", datetime.date(2025, 10, 13): 'Columbus Day'}

In [1662]:
data['is_holiday'] = [1 if x else 0 for x in data['data_as_of'].dt.date.isin(us_holidays)]

data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,...,day_sin,day_cos,week_of_year,week_num_sin,week_num_cos,month,month_num,month_sin,month_cos,is_holiday
0,18.01,855.0,0,2025-03-19 08:43:10,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,18.01,855.0,8,...,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0
1,15.53,688.0,0,2025-03-19 08:43:10,"40.67445,-73.80181 40.67374,-73.80154 40.67261...",Queens,VWE S MP2.66 (Exit 2 Rockaway Blvd) - Belt Pkw...,15.53,688.0,8,...,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0
2,15.53,688.0,-101,2025-03-19 08:43:10,"40.6167105,-74.15242 40.61572,-74.15276 40.614...",Staten Island,SIE W - MLK N WOOLEY AVENUE - WLAKER STREET,15.53,688.0,8,...,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0
3,8.69,1358.0,0,2025-03-19 08:43:10,"40.66673,-73.78649 40.66642,-73.78958 40.66642...",Queens,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave,8.69,1358.0,8,...,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0
4,19.88,448.0,0,2025-03-19 08:43:10,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,19.88,448.0,8,...,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,18.01,490.0,0,2025-03-19 08:03:11,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,18.01,490.0,8,...,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0
996,50.95,28.0,0,2025-03-19 08:03:11,"40.6077805,-74.14091 40.60826,-74.132101",Staten Island,SIE W BRADLEY AVENUE - WOOLEY AVENUE,50.95,28.0,8,...,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0
997,14.91,1023.0,0,2025-03-19 08:03:11,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,14.91,1023.0,8,...,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0
998,14.91,1023.0,-101,2025-03-19 08:03:11,"40.81376,-73.93128 40.8127206,-73.9316 40.8121...",Bronx,MDE N RFK Bridge - 142nd St,14.91,1023.0,8,...,0.974928,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0


**Creating The Non-Business Day Column**

In [1663]:
data['is_non_business_day'] = [1 if x > 4 or y else 0 for x, y in zip(data['day_of_week_num'], data['is_holiday'])]
data

,speed,travel_time,status,data_as_of,link_points,borough,link_name,speed_filled,travel_time_filled,hour,...,day_cos,week_of_year,week_num_sin,week_num_cos,month,month_num,month_sin,month_cos,is_holiday,is_non_business_day
0,18.01,855.0,0,2025-03-19 08:43:10,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,18.01,855.0,8,...,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0,0
1,15.53,688.0,0,2025-03-19 08:43:10,"40.67445,-73.80181 40.67374,-73.80154 40.67261...",Queens,VWE S MP2.66 (Exit 2 Rockaway Blvd) - Belt Pkw...,15.53,688.0,8,...,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0,0
2,15.53,688.0,-101,2025-03-19 08:43:10,"40.6167105,-74.15242 40.61572,-74.15276 40.614...",Staten Island,SIE W - MLK N WOOLEY AVENUE - WLAKER STREET,15.53,688.0,8,...,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0,0
3,8.69,1358.0,0,2025-03-19 08:43:10,"40.66673,-73.78649 40.66642,-73.78958 40.66642...",Queens,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave,8.69,1358.0,8,...,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0,0
4,19.88,448.0,0,2025-03-19 08:43:10,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,19.88,448.0,8,...,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,18.01,490.0,0,2025-03-19 08:03:11,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,18.01,490.0,8,...,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0,0
996,50.95,28.0,0,2025-03-19 08:03:11,"40.6077805,-74.14091 40.60826,-74.132101",Staten Island,SIE W BRADLEY AVENUE - WOOLEY AVENUE,50.95,28.0,8,...,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0,0
997,14.91,1023.0,0,2025-03-19 08:03:11,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,14.91,1023.0,8,...,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0,0
998,14.91,1023.0,-101,2025-03-19 08:03:11,"40.81376,-73.93128 40.8127206,-73.9316 40.8121...",Bronx,MDE N RFK Bridge - 142nd St,14.91,1023.0,8,...,-0.222521,12,0.98904,0.147647,March,3,1.0,6.123234e-17,0,0


In [1664]:
# data['is_school_day'] --> based on the region we choose

**Dropping Unnecessary Columns**

In [1665]:
data.columns

Index(['speed', 'travel_time', 'status', 'data_as_of', 'link_points',
       'borough', 'link_name', 'speed_filled', 'travel_time_filled', 'hour',
       'minute', 'minute_number', 'minute_sin', 'minute_cos', 'day_of_week',
       'day_of_week_num', 'day_sin', 'day_cos', 'week_of_year', 'week_num_sin',
       'week_num_cos', 'month', 'month_num', 'month_sin', 'month_cos',
       'is_holiday', 'is_non_business_day'],
      dtype='object')

In [1666]:
data = data.drop(['speed_filled', 'travel_time_filled', 'status', 'hour', 'minute', 'minute_number', 'day_of_week', 'day_of_week_num', 'week_of_year', 'month'], axis = 1)
data.columns

Index(['speed', 'travel_time', 'data_as_of', 'link_points', 'borough',
       'link_name', 'minute_sin', 'minute_cos', 'day_sin', 'day_cos',
       'week_num_sin', 'week_num_cos', 'month_num', 'month_sin', 'month_cos',
       'is_holiday', 'is_non_business_day'],
      dtype='object')

In [1667]:
data

,speed,travel_time,data_as_of,link_points,borough,link_name,minute_sin,minute_cos,day_sin,day_cos,week_num_sin,week_num_cos,month_num,month_sin,month_cos,is_holiday,is_non_business_day
0,18.01,855.0,2025-03-19 08:43:10,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,0.757565,-0.652760,0.974928,-0.222521,0.98904,0.147647,3,1.0,6.123234e-17,0,0
1,15.53,688.0,2025-03-19 08:43:10,"40.67445,-73.80181 40.67374,-73.80154 40.67261...",Queens,VWE S MP2.66 (Exit 2 Rockaway Blvd) - Belt Pkw...,0.757565,-0.652760,0.974928,-0.222521,0.98904,0.147647,3,1.0,6.123234e-17,0,0
2,15.53,688.0,2025-03-19 08:43:10,"40.6167105,-74.15242 40.61572,-74.15276 40.614...",Staten Island,SIE W - MLK N WOOLEY AVENUE - WLAKER STREET,0.757565,-0.652760,0.974928,-0.222521,0.98904,0.147647,3,1.0,6.123234e-17,0,0
3,8.69,1358.0,2025-03-19 08:43:10,"40.66673,-73.78649 40.66642,-73.78958 40.66642...",Queens,Belt Pkwy W JFK Expressway - VWE N Jamaica Ave,0.757565,-0.652760,0.974928,-0.222521,0.98904,0.147647,3,1.0,6.123234e-17,0,0
4,19.88,448.0,2025-03-19 08:43:10,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,0.757565,-0.652760,0.974928,-0.222521,0.98904,0.147647,3,1.0,6.123234e-17,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,18.01,490.0,2025-03-19 08:03:11,"40.7279205,-73.83298 40.7268904,-73.83239 40.7...",Queens,VWE S MP6.39 (Exit 11 Jewel Ave) - MP4.63 (Exi...,0.859406,-0.511293,0.974928,-0.222521,0.98904,0.147647,3,1.0,6.123234e-17,0,0
996,50.95,28.0,2025-03-19 08:03:11,"40.6077805,-74.14091 40.60826,-74.132101",Staten Island,SIE W BRADLEY AVENUE - WOOLEY AVENUE,0.859406,-0.511293,0.974928,-0.222521,0.98904,0.147647,3,1.0,6.123234e-17,0,0
997,14.91,1023.0,2025-03-19 08:03:11,"40.69158,-73.99929 40.69221,-73.999531 40.6928...",Brooklyn,BQE S LEONARD STREET - ATLANTIC AVENUE,0.859406,-0.511293,0.974928,-0.222521,0.98904,0.147647,3,1.0,6.123234e-17,0,0
998,14.91,1023.0,2025-03-19 08:03:11,"40.81376,-73.93128 40.8127206,-73.9316 40.8121...",Bronx,MDE N RFK Bridge - 142nd St,0.859406,-0.511293,0.974928,-0.222521,0.98904,0.147647,3,1.0,6.123234e-17,0,0


**Choosing A Suitable Borough**

In [1668]:
data[data['borough'] == 'Manhattan']['link_name'].unique()

array(['BKN Bridge Manhattan Side - FDR N Catherine Slip',
       'FDR N Catherine Slip - 25th St', '12th Ave S 57th St - 45th St',
       '11th ave n ganservoort - 12th ave @ 40th st',
       'BQE N Atlantic Ave - BKN Bridge Manhattan Side',
       '12th Ave N 40th - 57th St',
       'BQE N Atlantic Ave - MAN Bridge Manhattan Side',
       'TBB W - FDR S MANHATTAN TRUSS - E116TH STREET',
       '12th ave @ 45th - 11 ave ganservoort st', 'FDR N 25th - 63rd St',
       'LINCOLN TUNNEL E SOUTH TUBE - NJ - NY',
       'LINCOLN TUNNEL E CENTER TUBE NJ - NY',
       'LINCOLN TUNNEL W CENTER TUBE NY - NJ',
       'FDR N - TBB E 116TH STREET - MANHATTAN TRUSS',
       'FDR S 63rd - 25th St', 'Westside Hwy N 57th St - GWB',
       'BBT E Manhattan Portal - Toll Plaza',
       'FDR S Catherine Slip - BKN Bridge Manhattan Side',
       'FDR S 25th St - Catherine Slip',
       'LINCOLN TUNNEL W NORTH TUBE NY - NJ',
       '11th ave s ganservoort - west st @ spring st',
       'Westside Hwy S GWB 

In [1669]:
data['borough'].unique()

array(['Brooklyn', 'Queens', 'Staten Island', 'Bronx', 'Manhattan'],
      dtype=object)

In [1670]:
boroughs = data['borough'].unique()

for borough in boroughs:
  borough_data = data[data['borough'] == borough]
  print("The average speed in", borough, "=", borough_data['speed'].mean())

The average speed in Brooklyn = 21.827283950617282
The average speed in Queens = 30.779701986754972
The average speed in Staten Island = 39.32853773584906
The average speed in Bronx = 33.92035714285714
The average speed in Manhattan = 24.819999999999997


In [1671]:
boroughs = df['borough'].unique()

for borough in boroughs:
  borough_data = df[df['borough'] == borough]
  print("The average speed in", borough, "=", borough_data['speed'].mean())

The average speed in Brooklyn = 11.603580246913578
The average speed in Queens = 30.779701986754972
The average speed in Staten Island = 32.6236320754717
The average speed in Bronx = 30.436734693877554
The average speed in Manhattan = 14.249712918660284


In [1672]:
boroughs = data['borough'].unique()

for borough in boroughs:
  borough_data = data[data['borough'] == borough]
  print("The average travel time in", borough, "=", borough_data['travel_time'].mean())

The average travel time in Brooklyn = 444.7654320987654
The average travel time in Queens = 351.9006622516556
The average travel time in Staten Island = 156.1509433962264
The average travel time in Bronx = 247.28571428571428
The average travel time in Manhattan = 392.6555023923445


In [1673]:
boroughs = df['borough'].unique()

for borough in boroughs:
  borough_data = df[df['borough'] == borough]
  print("The average travel time in", borough, "=", borough_data['travel_time'].mean())

The average travel time in Brooklyn = 297.7283950617284
The average travel time in Queens = 351.9006622516556
The average travel time in Staten Island = 88.44339622641509
The average travel time in Bronx = 193.5765306122449
The average travel time in Manhattan = 284.99043062200957
